# 4.1 - Macrobond web API - Revision History

*Using Macrobond's web API features to query one or more time series in a point-in-time fashion. Revision history is defined as the feature allowing to request the original values of a time series before it could have been revised by the source. The vintageTimeStamp provides the point-in-time stamp as of which the series was known to have the values retrieved. This time stamp corresponds to when Macrobond has made the data available in its database.*

This notebook aims to provide examples of how to use Macrobond's web API call methods to work with Revision History.

We will focus here on using various calls from observing whether a time series carries Revision History up to requesting the full array of historical revisions.

*The examples uses the Web API, but you can chose to use the desktop COM API and get the same result. Full error handling is omitted for brevity*

***

## Importing packages

In [ ]:
# from oauthlib.oauth2 import BackendApplicationClient
# from requests_oauthlib import OAuth2Session
# import json
# import matplotlib.ticker as tck
# import pandas as pd
# import seaborn
# import plotly.express as px
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go
# import matplotlib.pyplot as plt
# import matplotlib.dates as md
# import getpass as gb
# import numpy as np
# import datetime as dt
# import time


import pandas
from matplotlib import pyplot

from macrobond_financial.web import WebClient

***

## Get the data - GetRevisionInfo

GetRevisionInfo provides high level information whether the time series store Revision History and had Revisions yet or not. The output also provides the list of vintageTimeStamps for series carrying Revision History.

In [ ]:
with WebClient() as api:
    data_frame = api.get_revision_info("gbgdp").to_pd_data_frame()
data_frame

Our time series is enabled to store Revision History and already had revisions. Our records start in 2016-03-31.
Let's now observe the most recent time stamps recorded.

In [ ]:
with WebClient() as api:
    info = api.get_revision_info("gbgdp").object()[0]
list(map(str,info.vintage_time_stamps))[-10:]

***

## Get the data - FetchObservationHistory

FetchObservationHistory finds the various values a specific historical release 'observationDate' could have had over time. The historical revisions are provided with their respective dates of integration 'timeStamps'.

In [ ]:
def downloadMbApi(request):
    url = "https://api.macrobondfinancial.com/v1/" + request
    # This is typically the first time when we do not yet have a token
    if not mbapi.authorized:
        mbapi.fetch_token(
            token_url=token_url, client_id=client_id, client_secret=client_secret
        )
    r = mbapi.get(url)
    if r.status_code != 401:
        return r
    # If authorization failed, it is likely that the token has expired. Get a new one and try again.
    mbapi.fetch_token(
        token_url=token_url, client_id=client_id, client_secret=client_secret
    )
    r = mbapi.get(url)
    return r


# We are fetching here the Q3 2020 observations for gbgdp (United Kingdom GDP).
d = downloadMbApi("series/fetchobservationhistory?n=gbgdp&t=2020-07-01")
x = d.json()

In [ ]:
df3 = pd.DataFrame((x), columns=["values", "timeStamps"])
df3 = df3.to_numpy().flatten()
data = pd.DataFrame()
for i in df3:
    df3 = pd.DataFrame(i)
    data = pd.concat([data, df3], axis=1)
data.columns = ["values", "timeStamps"]
data

**Review TO: Do we really want to quote specific observations and values here? The underlying data will change and the text will be wrong.**
As we have requested here the historical value of GB GDP accounting for Q3 2020, we can see that it was originally published on 2021-02-12 at 7AM UTC with a value of 490,861,000,000.
This Q3 2020 GB GDP was then further revised 3 times with the latest revision to date published on 2021-05-20 at 8.32AM UTC with a value of 498,429,000,000.

Let's measure the time difference from the first publication to the last revision:

In [ ]:
data.timeStamps = pd.to_datetime(data.timeStamps)
data.timeStamps.max() - data.timeStamps.min()

Let's plot our revisions

In [ ]:
plt.rcParams["figure.figsize"] = [16, 9]

fig, ax = plt.subplots()
ax.plot(
    data["timeStamps"],
    data["values"],
    color=(27 / 255, 54 / 255, 93 / 255),
    linewidth=4,
)
plt.title("Q3 2020 GB GDP historical revisions", fontsize=24)
plt.xlabel("Vintages", fontsize=14)
plt.ylabel("GB GDP", fontsize=14)
ax.grid()
plt.autoscale()
plt.show()

***

## Get the data - FetchNthReleaseSeries

FetchNthReleaseSeries retrieves the nth change of value of the requested time series.
Now that we have pulled out the revisions of a specific time series for a fixed observation via the FetchObservationHistory endpoint, we can isloate the nth revision in this array. This can be particularly useful to observe patterns of revisions from a single source or on a unique concept across various regions.  

In [ ]:
# We are fetching here the first revision (n=1) for each observation of gbgdp (United Kingdom GDP).
d = downloadMbApi("series/fetchnthreleaseseries?nth=1&getTimesOfChange=true&n=gbgdp")
x = d.json()

with WebClient(credentials.client_id, credentials.client_secret) as api:
    info = api.revision.get_revision_info("gbgdp").object()[0]
list(map(str,info.vintage_time_stamps))[-10:]

As we are fetching the first revision for each observation, let's bear in mind that the very first vintage recorded for this series was on 2016-03-31 i.e. there was no revision recorded beforehand hence we can expect to see no timesOfChange nor values prior to this date.

In [ ]:
df4 = pd.DataFrame((x), columns=["timesOfChange", "dates", "values"])
df4 = df4.to_numpy().flatten()
data = pd.DataFrame()
for i in df4:
    df4 = pd.DataFrame(i)
    data = pd.concat([data, df4], axis=1)
data.columns = ["timesOfChange", "dates", "values"]
data.dates = pd.to_datetime(data.dates, dayfirst=True)
data.sort_values(by="dates", ascending=False)

**Review TO: Do we really want to quote specific observations and values here? New values will be added to this series, and value 262 will no longer be listed above.**

We observe here on row 262 the timeOfChange = 2021-03-31T06:00:00Z and value 497,909,000,000 that we previously returned when using FetchObservationHistory on Q3 2020 for this series. 

***

## Get the data - FetchVintageSeries

In [ ]:
# We are fetching here the the gbgdp (United Kingdom GDP) as of 2020-06-30, which is the vintage we have been referring to before.
d = downloadMbApi(
    "series/fetchvintageseries?t=2020-07-01&getTimesOfChange=false&n=gbgdp"
)
x = d.json()

Note that we have inserted the time of the vintage to retrieve in the query = 2020-07-01 which is one day after the actual vintage in order to make sure we fall back on the most recent one prior to this date, which is 2020-06-30. We also could have been using a time stamp just after 06:00:00Z as this is when the vintage was recorded. 

In [ ]:
df5 = pd.DataFrame((x), columns=["vintageTimeStamp"])
df5

Let's now observe the time series as it was as of that date:

In [ ]:
df6 = pd.DataFrame((x), columns=["dates", "values"])
df6 = df6.to_numpy().flatten()
data_pit = pd.DataFrame()
for i in df6:
    df6 = pd.DataFrame(i)
    data_pit = pd.concat([data_pit, df6], axis=1)
data_pit.columns = ["dates", "values"]
data_pit

Let's now plot our time series as of this specific vintage time stamp:

In [ ]:
dates_h = pd.to_datetime(data_pit.dates)

plt.rcParams["figure.figsize"] = [16, 9]

fig, ax = plt.subplots()
ax.plot(dates_h, data_pit["values"], color=(27 / 255, 54 / 255, 93 / 255), linewidth=4)
plt.title("GB GDP as of 2020-06-30", fontsize=24)
plt.xlabel("Dates", fontsize=14)
plt.ylabel("GB GDP", fontsize=14)
ax.grid()
plt.autoscale()
plt.show()

***

## Get the data - FetchAllVintageSeries

FetchAllVintageSeries retrieves the full array of revisions over time for the requested time series.
We can build a dataframe of such revisions. 

In [ ]:
# We are fetching here the the gbgdp (United Kingdom GDP) as of 2020-06-30, which is the vintage we have been referring to before.
d = downloadMbApi("series/fetchallvintageseries?n=gbgdp")
x = d.json()

In [ ]:
data = pd.DataFrame(x)
s = data
data.vintageTimeStamp = pd.to_datetime(data.vintageTimeStamp, dayfirst=True)


vintageTimeStamp = data["vintageTimeStamp"].values
data = data.drop(["metadata", "vintageTimeStamp"], axis=1)
df1 = data.to_numpy().flatten()
# ----------------------------------------
df = pd.DataFrame()
for i in df1:
    df2 = pd.DataFrame(i)
    df = pd.concat([df, df2], axis=1)
i = 1
sn = []
for j in range(len(df.columns) // 2):
    series1 = pd.DataFrame(df.iloc[:, i - 1])
    series2 = pd.DataFrame(df.iloc[:, i])
    sn1 = pd.concat([series1, series2], axis=1)
    l = str(vintageTimeStamp[(i - 1) // (2)])
    sn1.columns = ["dates", l]
    sn1 = sn1.set_index("dates")
    sn1.index.name = None
    sn1 = sn1.squeeze()
    sn1 = sn1[np.logical_not(np.isnan(sn1))]
    sn.append(sn1)
    i = i + 2

dff = pd.DataFrame(sn)
data = dff.T
data